# 한글 문서를 위한 스키마 정의 후 Vector Store 생성 

---

---

# 1. Bedrock Client 생성

In [3]:
! pip list | grep langchain # 0.0.312
! pip list | grep opensearch # 2.3.2
! pip list | grep pypdf

langchain                                0.0.338
opensearch-py                            2.3.2
pypdf                                    3.17.1


In [4]:
%load_ext autoreload
%autoreload 2

import sys, os
# module_path = "../../../utils"
# sys.path.append(os.path.abspath(module_path))
# print(os.path.abspath(module_path))

module_path = "./utils"
sys.path.append(os.path.abspath(module_path))
print(os.path.abspath(module_path))

/root/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/02_qa_chatbot/01_preprocess_docs/utils


In [5]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print(colored("\n== FM lists ==", "green"))
pprint(bedrock_info.get_list_fm_models())

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Command': 'cohere.command-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'TBD'}


# 2. Titan Embedding 모델 로딩

In [6]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

llm_emb = BedrockEmbeddings(client=boto3_bedrock)
llm_emb

BedrockEmbeddings(client=<botocore.client.BedrockRuntime object at 0x7fca0bdfd570>, region_name=None, credentials_profile_name=None, model_id='amazon.titan-embed-text-v1', model_kwargs=None, endpoint_url=None)

# 3. Load all Json files

In [7]:
from utils.proc_docs import get_load_json, show_doc_json

In [8]:
import glob

# Specify the directory and file pattern for .txt files
folder_path = 'data/poc/preprocessed_json/*ko.json'

# List all .txt files in the specified folder
json_files = glob.glob(folder_path)
# json_files = ['data/poc/customer_EFOTA.json']

# Load each item per json file and append to a list
doc_json_list = []
for file_path in json_files:
    doc_json = get_load_json(file_path)
    doc_json_list.append(doc_json)

print("all json files: ", len(doc_json_list))    
# Flatten the list of lists into a single list
all_docs = []
for item in doc_json_list:
        all_docs.extend(item)
        
print("all items: ", len(all_docs))

all json files:  1
all items:  1732


In [10]:
all_docs[0:2]

[Document(page_content='AR 데모 바코드.다음 바코드를 사용하여 녹스 캡처 AR 기능을 테스트하세요.바코드 1 바코드 2 바코드 3 바코드 4 바코드 5 바코드 6', metadata={'source': 'all_processed_data_ko.json', 'seq_num': 1, 'title': 'AR 데모 바코드', 'url': 'https://docs.samsungknox.com/admin/knox-capture/ar-demo-barcodes', 'project': 'KCAP', 'last_updated': '2023-10-16'}),
 Document(page_content='동영상.이 섹션에는 Knox Capture와 관련된 제품 및 사용 방법 비디오가 포함되어 있습니다.삼성 Knox Capture 시작하기 이 비디오에서는 삼성 Knox Capture를 사용하여 모바일 디바이스를 강력한 바코드 스캐너로 변환하여 바코드 데이터를 읽고, 처리하고, 다른 애플리케이션으로 출력하는 방법을 보여 드리겠습니다.삼성 Knox Capture: Galaxy 디바이스의 엔터프라이즈급 모바일 스캔 솔루션 이 제품 소개 비디오는 Galaxy XCover Pro와 같은 견고한 삼성 디바이스를 엔터프라이즈급 바코드 스캐너로 전환하는 것이 얼마나 쉬운지 보여줍니다.', metadata={'source': 'all_processed_data_ko.json', 'seq_num': 2, 'title': '동영상', 'url': 'https://docs.samsungknox.com/admin/knox-capture/how-to-videos', 'project': 'KCAP', 'last_updated': '2023-07-26'})]

# 4. Chunking JSON Doc 

## Chunk Size and Chunk Overlap Size 결정

In [11]:
chunk_size = 1024
chunk_overlap = 256


## Chunking

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SpacyTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
    separators=["\n\n", "\n", ".", " ", ""],
    length_function = len,
)

chunk_docs = text_splitter.split_documents(all_docs)
print(f"Number of chunk_docs after split and chunking= {len(chunk_docs)}")

Number of chunk_docs after split and chunking= 4057


In [13]:
chunk_docs[0:1]

[Document(page_content='AR 데모 바코드.다음 바코드를 사용하여 녹스 캡처 AR 기능을 테스트하세요.바코드 1 바코드 2 바코드 3 바코드 4 바코드 5 바코드 6', metadata={'source': 'all_processed_data_ko.json', 'seq_num': 1, 'title': 'AR 데모 바코드', 'url': 'https://docs.samsungknox.com/admin/knox-capture/ar-demo-barcodes', 'project': 'KCAP', 'last_updated': '2023-10-16'})]

# 5. Index 생성

## Index 이름 결정

In [14]:
index_name = "genai-poc-knox-incremental-ko-1024c-256o-v8"

## Index 스키마 정의

In [15]:
index_body = {
    'settings': {
        'analysis': {'analyzer': {'my_analyzer': {'char_filter': ['html_strip'],
                                                    'tokenizer': 'nori',
                                                       'filter': [
                                                                   'nori_number',
                                                                   'lowercase',
                                                                   'trim',
                                                                   'my_nori_part_of_speech'
                                                                 ],
                                                       'type': 'custom'}},
                                   'tokenizer': {'nori': {
                                                  'decompound_mode': 'mixed',
                                                  'discard_punctuation': 'true',
                                                  'type': 'nori_tokenizer'}
                                                },
                                    "filter": {
                                          "my_nori_part_of_speech": {
                                                "type": "nori_part_of_speech",
                                                "stoptags": [
                                                      "E", "IC", "J", "MAG", "MAJ",
                                                      "MM", "SP", "SSC", "SSO", "SC",
                                                      "SE", "XPN", "XSA", "XSN", "XSV",
                                                      "UNA", "NA", "VSV"
                                                ]
                                          }
                                    }
                    },        
        'index': {
            'knn': True,
            'knn.space_type': 'cosinesimil'  # Example space type
        }
    },
    'mappings': {
        'properties': {
            'metadata': {
                'properties': {
                               'source' : {'type': 'keyword'},                    
                               'last_updated': {'type': 'date'},
                               'project': {'type': 'keyword'},
                               'seq_num': {'type': 'long'},
                               'title': {'type': 'text'},  # For full-text search
                               'url': {'type': 'text'},  # For full-text search
                            }
            },            
            'text': {
                'analyzer': 'my_analyzer',
                'search_analyzer': 'my_analyzer',
                'type': 'text'
            },
            'vector_field': {
                'type': 'knn_vector',
                'dimension': 1536  # Replace with your vector dimension
            }
        }
    }
}


# 5. LangChain OpenSearch VectorStore 생성 
## 선수 조건


## 오픈 서치 도메인 및 인증 정보 세팅

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.

In [16]:
from utils.proc_docs import get_parameter

In [17]:
import boto3
ssm = boto3.client('ssm', 'us-east-1')

opensearch_domain_endpoint = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_domain_endpoint',
)

opensearch_user_id = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_userid',
)

opensearch_user_password = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_password',
)


In [18]:
opensearch_domain_endpoint = opensearch_domain_endpoint
rag_user_name = opensearch_user_id
rag_user_password = opensearch_user_password

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

## OpenSearch Client 생성

In [19]:
from utils.opensearch import opensearch_utils

In [20]:
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

## 오픈 서치 인덱스 생성 
- 오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [21]:
from utils.opensearch import opensearch_utils

In [22]:

index_exists = opensearch_utils.check_if_index_exists(
    os_client,
    index_name
)

if index_exists:
    opensearch_utils.delete_index(
        os_client,
        index_name
    )

opensearch_utils.create_index(os_client, index_name, index_body)
index_info = os_client.indices.get(index=index_name)
print("Index is created")
pprint(index_info)

index_name=genai-poc-knox-incremental-ko-1024c-256o-v8, exists=True

Deleting index:
{'acknowledged': True}

Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'genai-poc-knox-incremental-ko-1024c-256o-v8'}
Index is created
{'genai-poc-knox-incremental-ko-1024c-256o-v8': {'aliases': {},
                                                 'mappings': {'properties': {'metadata': {'properties': {'last_updated': {'type': 'date'},
                                                                                                         'project': {'type': 'keyword'},
                                                                                                         'seq_num': {'type': 'long'},
                                                                                                         'source': {'type': 'keyword'},
                                                                                                         'title': {'type': 'text'},
        

## 랭체인 인덱스 연결 오브젝트 생성

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

In [23]:
from langchain.vectorstores import OpenSearchVectorSearch

In [24]:
vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2",
    bulk_size=100000,
    timeout=60    
)
vector_db

## OpenSearch 에 문서 삽입

In [25]:
%%time

vector_db.add_documents(documents = chunk_docs, 
                        vector_field = "vector_field",
                        bulk_size = 1000000)


CPU times: user 11.7 s, sys: 603 ms, total: 12.3 s
Wall time: 7min 15s


['1cb0f4a4-97be-43f9-8750-bddd918ed55e',
 '4f7a3e2c-13af-451b-a8e4-5e3132a68ec4',
 '24a463c6-50ef-426a-8b95-b9a18abab371',
 'f17d43b6-3cd2-4f14-a325-868a223244af',
 '6b993717-b06c-4dd0-bdf8-1ce8ea521750',
 'ee1d5237-04d4-4d66-a07d-10cc4de93cb7',
 '7a9e07cd-64ac-4d18-9243-cf5849d4e227',
 'df818ff9-ce5e-4692-8c2b-f8c207923629',
 '1007ee43-aaff-47fb-a945-d19231c557f5',
 '591770ff-fe86-43de-9937-0ae5cbca6bd8',
 'ccf9691c-ebbd-4d77-97a8-ffc5812c1d6a',
 'ff5c9e66-1a03-4be3-9a19-c87aafa91c0a',
 'e17dea3d-be72-4050-bac6-81110048562d',
 'eff83210-3b65-4217-aab9-36b6314ef9ed',
 '2a4ce7ee-c7e1-49c7-8caf-698ec9909c33',
 'f060c308-3eae-4ae1-91d2-81935df7f42f',
 '8edd44cf-2044-4436-b9b5-4c0e6ced1f31',
 'bdce492d-f738-4f3f-a48a-48968e00a5cb',
 '15e291a9-985d-440b-aa29-6a11009fd961',
 'a31049e6-d7a4-4f57-a8c6-91795bda7bf1',
 'a382d6fb-129f-49c3-88aa-045c8bf8daa7',
 '3e59c079-1e43-44a4-a403-381860e4b4a4',
 '78c54a87-2271-4695-85bf-9655a8d3cdb2',
 'b40cc18e-825d-4c09-91de-57ed91da2118',
 'bd44f012-4c5e-

# 6. 검색 테스트

## Lexical 검색

In [26]:
query = "이미지 추가 하는 방법"
 #query = "how to add image"
query = opensearch_utils.get_query(
    query=query
)

print("query: ", query)
response = opensearch_utils.search_document(os_client, query, index_name)
opensearch_utils.parse_keyword_response(response, show_size=3)

query:  {'query': {'bool': {'must': [{'match': {'text': {'query': '이미지 추가 하는 방법', 'minimum_should_match': '0%', 'operator': 'or'}}}], 'filter': []}}}
# of searched docs:  10
# of display: 3
---------------------
_id in index:  30609e82-853d-412c-b7df-9664a1e6f565
9.5869665
.이미지를 추가하지 않으면 기본 일반 이미지가 표시됩니다.이미지 크기는 PNG 또는 JPG 형식일 수 있으며 크기는 2MB를 초과할 수 없습니다.액세스 포인트 - 선택적으로 디바이스 사용자가 Knox Configure 클라이언트를 열 수 있는 추가 방법을 구성하세요.무시할 수 없는 알림 추가 - 디바이스 사용자가 탭하여 Knox Configure 클라이언트에 빠르게 액세스할 수 있는 영구 알림을 추가하려면 이 옵션을 선택합니다.애플리케이션 이름 - 알림에 표시된 애플리케이션 이름을 입력합니다.알림 메시지 - 알림에 표시된 메시지를 입력합니다.Knox 구성 포털은 다음과 같은 특수 문자를 지원하지 않습니다. #/$\ *% ^ &\\ () +?[].기기 프로필을 생성하기 위해 문자를 입력할 때는 이러한 요소가 사용되지 않도록 하십시오.애플리케이션 선택: 애플리케이션 선택 탭에서는 엄선된 관련 Tizen 기반 웨어러블 기기 앱 또는 사용자 지정 시계 모드 앱을 업로드하여 기기 프로필에 활용할 수 있습니다.또한 하드웨어 키를 다시 매핑한 후 호출할 수 있는 앱을 추가할 수도 있습니다.앱 업로드 및 추가 방법에 대한 추가 지침은 앱 관리 기능을 참조하세요.튜토리얼 진행 2/4 단계를 완료했습니다!다음 단계로 이동하여 Knox Configure 프로필을 다음과 같이 사용자 지정하세요. 1.설치 에디션 2.다이나믹 에디션: 일반 모드, 프로키오스크 모드
{'source': 'all_proce

## 시멘틱 검색

In [27]:
vector_db.similarity_search("이미지 추가 하는 방법")

[Document(page_content='.등록에 QR 코드를 사용하는 방법에 대한 자세한 내용은 프로필 생성 장치 및 장치 사용자 권한 향상을 참조하십시오. 이 개선 사항은 더 나은 권한 관리를 위해 역할 만들기 화면의 장치 및 업로드 부분에 있는 프로필로 할당 및 태그 관리 역할을 결합합니다.사용자에게 프로필로 할당 및 태그 관리 권한이 활성화되지 않은 경우, 장치 구성 팝업 (장치 > 업로드 화면의 작업 드롭다운 메뉴에서 사용 가능) 에서 프로필 지우기를 제외한 모든 MDM 프로필 옵션이 비활성화됩니다.기존 태그를 추가하고 덮어쓰는 기능은 대량 구성 화면에서 비활성화됩니다.추가 KME 국가 지원 미국 지역에서 KME 지원 국가로 추가되었습니다. 코스타리카, 도미니카공화국, 엘살바도, 과테말라, 온두라스, 니카라과, 푸에르토리코 이러한 국가는 KME 삼성 관리자 포털과 KME 가입 시 samsungknox.com 드롭다운 메뉴에서 지원 국가로 추가됩니다.릴리스 노트로 돌아가기', metadata={'source': 'all_processed_data_ko.json', 'seq_num': 500, 'title': '녹스 모바일 등록 1.31 릴리즈 노트', 'url': 'https://docs.samsungknox.com/admin/knox-mobile-enrollment/release-notes/20-05', 'project': 'KME', 'last_updated': '2023-07-26'}),
 Document(page_content='사용자 그룹에 사용자를 추가합니다.사용자 그룹을 생성한 후 사용자를 추가할 수 있습니다.사용자 그룹에 사용자를 추가하려면 다음 단계를 완료하세요. 1.그룹으로 이동합니다. 2.그룹 페이지에서 사용자를 추가하려는 특정 사용자 그룹 이름을 클릭합니다.그룹 유형은 사용자여야 합니다. 3.그룹 세부 정보 페이지에서 사용자 탭을 엽니다. 4.추가를 클릭합니다. 5.사용자 선택 화면에서 추가하려는 사용자를 선택한 다음 추가를 클릭

## 하이브리드 검색

In [28]:
from langchain.chains.question_answering import load_qa_chain
from utils.rag import get_semantic_similar_docs, get_lexical_similar_docs, get_ensemble_results

In [29]:
import copy
from langchain.schema import Document
from langchain import PromptTemplate
from operator import itemgetter

In [30]:
from utils.proc_docs import search_hybrid

In [32]:
%%time


filter01 = "[]"
filter02 = "[]"

# query = "how to add image"
query = "이미지 추가 하는 방법"

search_hybrid_result = search_hybrid(
    query=query,
    vector_db=vector_db,
    k=3,
    index_name= index_name,
    os_client=os_client,
    filter=[
        {"term": {"metadata.type": filter01}},
        {"term": {"metadata.source": filter02}},
    ],
    Semantic_Search = False,    
    Lexical_Search = False,    
    Hybrid_Search = True,     
    minimum_should_match = 75,   
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"]
    ensemble_weights=[.5, .5], # 시멘트 서치에 가중치 0.5 , 키워드 서치 가중치 0.5 부여.
    verbose=True
)



Query: 
 이미지 추가 하는 방법
##############################
similar_docs_semantic
##############################

Score: 1.0
['.등록에 QR 코드를 사용하는 방법에 대한 자세한 내용은 프로필 생성 장치 및 장치 사용자 권한 향상을 참조하십시오. 이 개선 사항은 더 나은 권한 관리를 위해 역할 만들기 화면의 장치 및 업로드 부분에 있는 프로필로 할당 및 태그 관리 역할을 결합합니다.사용자에게 프로필로 할당 및 태그 관리 권한이 활성화되지 않은 경우, 장치 구성 팝업 (장치 > 업로드 화면의 작업 드롭다운 메뉴에서 사용 가능) 에서 프로필 지우기를 제외한 모든 MDM 프로필 옵션이 비활성화됩니다.기존 태그를 추가하고 덮어쓰는 기능은 대량 구성 화면에서 비활성화됩니다.추가 KME 국가 지원 미국 지역에서 KME 지원 국가로 추가되었습니다. 코스타리카, 도미니카공화국, 엘살바도, 과테말라, 온두라스, 니카라과, 푸에르토리코 이러한 국가는 KME 삼성 관리자 포털과 KME 가입 시 samsungknox.com 드롭다운 메뉴에서 지원 국가로 추가됩니다.릴리스 노트로 돌아가기']
{'source': 'all_processed_data_ko.json', 'seq_num': 500, 'title': '녹스 모바일 등록 1.31 릴리즈 노트', 'url': 'https://docs.samsungknox.com/admin/knox-mobile-enrollment/release-notes/20-05', 'project': 'KME', 'last_updated': '2023-07-26'}
--------------------------------------------------

Score: 0.9987955041056603
['사용자 그룹에 사용자를 추가합니다.사용자 그룹을 생성한 후 사용자를 추가할 수 있습니다.사용자 그룹에 사용자를 추가하려면 다음 단계를 완료하세요. 1.그룹으로 이동합니다.

# 7. 검증 인덱스 생성

## Index 이름 결정

In [33]:
eval_index_name = "genai-poc-knox-kr-eval-sample-0.02-1024c-256o-v9"

## Sampling

In [34]:
import random
def get_sampling_doc(seed, ratio, docs):

    random.seed(seed)
    
    eval_docs = docs[:int(len(docs)*ratio)]
    
    return eval_docs
    
eval_docs = get_sampling_doc(seed=200, ratio=0.02, docs= all_docs)
print("eval docs: ", len(eval_docs))
eval_docs[0:2]
    
    

eval docs:  34


[Document(page_content='AR 데모 바코드.다음 바코드를 사용하여 녹스 캡처 AR 기능을 테스트하세요.바코드 1 바코드 2 바코드 3 바코드 4 바코드 5 바코드 6', metadata={'source': 'all_processed_data_ko.json', 'seq_num': 1, 'title': 'AR 데모 바코드', 'url': 'https://docs.samsungknox.com/admin/knox-capture/ar-demo-barcodes', 'project': 'KCAP', 'last_updated': '2023-10-16'}),
 Document(page_content='동영상.이 섹션에는 Knox Capture와 관련된 제품 및 사용 방법 비디오가 포함되어 있습니다.삼성 Knox Capture 시작하기 이 비디오에서는 삼성 Knox Capture를 사용하여 모바일 디바이스를 강력한 바코드 스캐너로 변환하여 바코드 데이터를 읽고, 처리하고, 다른 애플리케이션으로 출력하는 방법을 보여 드리겠습니다.삼성 Knox Capture: Galaxy 디바이스의 엔터프라이즈급 모바일 스캔 솔루션 이 제품 소개 비디오는 Galaxy XCover Pro와 같은 견고한 삼성 디바이스를 엔터프라이즈급 바코드 스캐너로 전환하는 것이 얼마나 쉬운지 보여줍니다.', metadata={'source': 'all_processed_data_ko.json', 'seq_num': 2, 'title': '동영상', 'url': 'https://docs.samsungknox.com/admin/knox-capture/how-to-videos', 'project': 'KCAP', 'last_updated': '2023-07-26'})]

In [35]:
chunk_docs = text_splitter.split_documents(eval_docs)
print(f"Number of chunk_docs after split and chunking= {len(chunk_docs)}")

Number of chunk_docs after split and chunking= 87


## 오픈 서치 인덱스 유무에 따라 삭제
오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [36]:
index_exists = opensearch_utils.check_if_index_exists(
    os_client,
    eval_index_name
)

if index_exists:
    opensearch_utils.delete_index(
        os_client,
        eval_index_name
    )
    
opensearch_utils.create_index(os_client, eval_index_name, index_body)
index_info = os_client.indices.get(index=eval_index_name)
print("Index is created")
pprint(index_info)    

index_name=genai-poc-knox-kr-eval-sample-0.02-1024c-256o-v9, exists=True

Deleting index:
{'acknowledged': True}

Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'genai-poc-knox-kr-eval-sample-0.02-1024c-256o-v9'}
Index is created
{'genai-poc-knox-kr-eval-sample-0.02-1024c-256o-v9': {'aliases': {},
                                                      'mappings': {'properties': {'metadata': {'properties': {'last_updated': {'type': 'date'},
                                                                                                              'project': {'type': 'keyword'},
                                                                                                              'seq_num': {'type': 'long'},
                                                                                                              'source': {'type': 'keyword'},
                                                                                                         

## 검증 인덱스 생성

In [37]:
eval_vector_db = OpenSearchVectorSearch(
    index_name= eval_index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2",
    bulk_size=100000,
    timeout=60    
)
vector_db

In [38]:
%%time

eval_vector_db.add_documents(documents = chunk_docs, 
                        vector_field = "vector_field",
                        bulk_size = 1000000)


CPU times: user 265 ms, sys: 10.6 ms, total: 276 ms
Wall time: 9.61 s


['6a708186-a75b-40af-b671-e83e11cf0065',
 '6f3edade-ca32-4b7a-b7a3-279df9caed5c',
 '897f21a4-b289-4d31-a4ce-c077a45ebac3',
 '0857170a-3948-4f47-a331-51af307b638c',
 'ccead5c5-0160-4061-bd1c-553b63da1b33',
 'e317aa32-c9e6-4694-bdde-586ace8de620',
 'b02e67f1-662c-4c69-8f10-56f4bd2b9f04',
 'bb15fc61-0389-4f65-a45a-9e41667d0e46',
 '86c3cdf7-d389-44b4-82f8-fb39a0593556',
 '5cd1d74c-f95c-4f8b-a48f-25e3bba3a3d1',
 '293588fe-d5da-4f0d-96d5-d98b3032e186',
 '676610d3-1bda-48b3-9275-7a1af1b26d5f',
 'eaa88cc8-b8f7-4cc5-93b8-d65d55a8e2f1',
 '897501b1-9b01-4a93-ba2b-73da446b81f2',
 'e40fa4f2-15ee-43ed-aea6-3a0fbb70454a',
 '71453887-92f5-4757-ae1c-b94023c30f58',
 '3226f5f0-32a7-4109-8449-eb49aafcba57',
 'a3c61cb7-9465-438c-9b96-89f9bd8f787f',
 'c1f2b7ee-384d-4522-becb-108c1a09466c',
 '32692677-91ba-4daf-ab5e-5386f848e96c',
 'c7d35c86-42db-4ded-93fb-42493a1db3f2',
 'a5fcb8b6-cd78-46b9-9bf2-65244c5bf032',
 '00b9cd98-0be4-4b5f-beb6-a68f69774c88',
 '2ed6f1f1-193e-42ac-a8d3-bd8b9721afde',
 '8ab3b556-6d4a-

In [39]:
%%time


filter01 = "[]"
filter02 = "[]"

# query = "how to add image"
query = "이미지 추가 하는 방법"

search_hybrid_result = search_hybrid(
    query=query,
    vector_db= eval_vector_db,
    k=3,
    index_name= eval_index_name,
    os_client=os_client,
    filter=[
        {"term": {"metadata.type": filter01}},
        {"term": {"metadata.source": filter02}},
    ],
    Semantic_Search = False,    
    Lexical_Search = False,    
    Hybrid_Search = True,     
    minimum_should_match = 75,   
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"]
    ensemble_weights=[.5, .5], # 시멘트 서치에 가중치 0.5 , 키워드 서치 가중치 0.5 부여.
    verbose=True
)



Query: 
 이미지 추가 하는 방법
##############################
similar_docs_semantic
##############################

Score: 1.0
['.Knox Capture는 구성을 가져오고 성공 메시지를 표시합니다. QR 코드가 포함된 #Import 메시지를 표시합니다. QR 코드를 통해 구성을 공유한 경우 다음과 같이 구성을 가져올 수 있습니다. 1.구성 가져오기 메뉴에서 QR 코드 아이콘을 탭하여 카메라 스캐너를 시작합니다. 2.카메라를 QR 코드 쪽으로 향하게 합니다. 3.QR 코드를 스캔한 후 Knox Capture는 자동으로 구성을 가져오고 성공 메시지를 표시합니다.구성이 QR 코드 하나에 비해 너무 큰 경우 Knox Capture는 여러 QR 코드를 PDF 파일에 표시합니다.최종 사용자가 가져오기 프로세스를 완료하려면 모두 스캔해야 합니다.사용자가 동일한 구성에 속하지 않는 QR 코드를 스캔하려고 하면 다른 QR 코드를 스캔할 때까지 오버레이에 오류 메시지가 표시됩니다.']
{'source': 'all_processed_data_ko.json', 'seq_num': 14, 'title': '구성 내보내기 및 배포', 'url': 'https://docs.samsungknox.com/admin/knox-capture/how-to-guides/export-and-deploy-configuration', 'project': 'KCAP', 'last_updated': '2023-07-26'}
--------------------------------------------------

Score: 0.9868393820711998
['.#Visual cue 시각적 신호는 카메라 스캔 UI의 중심에 있는 뷰파인더 오버레이로, 사용자의 주의를 바코드에 집중시킵니다.비주얼 큐의 크기는 실제 스캔 영역의 크기와 같습니다.설정 옵션 보기 파인더 선택 직사각형 (기본값) — 감지된 바코드 주위에 직사각

# A. Reference

- [Building a RAG AI with OpenSearch Serverless and LangChain](https://caylent.com/blog/building-a-rag-with-open-search-serverless-and-lang-chain)